# Machine Learning

Analisis basico de los datos en el dataframe y aplicacion de un par de algoritmos de machine learning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error


In [2]:
df = pd.read_csv('/home/matias/Escritorio/ML/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196719 entries, 0 to 196718
Data columns (total 27 columns):
id                            196719 non-null object
created_on                    196719 non-null object
operation                     196719 non-null object
property_type                 196719 non-null object
place_name                    196670 non-null object
place_with_parent_names       196719 non-null object
country_name                  196719 non-null object
state_name                    196719 non-null object
geonames_id                   164653 non-null float64
lat-lon                       130239 non-null object
lat                           130239 non-null float64
lon                           130239 non-null float64
price                         165687 non-null float64
currency                      165687 non-null object
price_aprox_local_currency    165687 non-null float64
price_aprox_usd               165687 non-null float64
surface_total_in_m2           14667

In [3]:
# Primero separamos los datos que nos interesan analizar para este primer analisis decidi tomar
#  place_name property_type rooms

train = df[['property_type','surface_total_in_m2', 'place_name', 'rooms','price']]
train =train[train['place_name'].notnull()]
train.head()

,property_type,surface_total_in_m2,place_name,rooms,price
0,house,335.0,Merlo,NaN,1199000.0
1,house,0.0,Pinamar,NaN,NaN
2,house,526.0,Pinamar,NaN,200000.0
3,house,312.0,Palermo Viejo,NaN,2000000.0
4,apartment,0.0,Puerto Madero,NaN,570000.0


In [4]:
# Rellenamos la superficie faltante con el promedio por barrios
train['surface_total_in_m2'] = train.groupby('place_name')['surface_total_in_m2'].transform(lambda x: x.fillna(x.median()))

In [5]:
# Podriamos completar los valores nulos pero por ahora vamos a dropear los nulos
train.dropna(inplace=True)
train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 108657 entries, 30 to 196718
Data columns (total 5 columns):
property_type          108657 non-null object
surface_total_in_m2    108657 non-null float64
place_name             108657 non-null object
rooms                  108657 non-null float64
price                  108657 non-null float64
dtypes: float64(3), object(2)
memory usage: 5.0+ MB


In [24]:
len(train)

108657

In [25]:
train. head()

,property_type,surface_total_in_m2,place_name,rooms,price
30,house,300.0,Mataderos,5.0,185000.0
31,apartment,199.0,Pilar,2.0,97000.0
32,store,265.0,San Isidro,1.0,750000.0
34,house,138.5,San Martín de los Andes,4.0,480000.0
35,house,138.5,San Martín de los Andes,3.0,150000.0


In [31]:
test = pd.read_csv('/home/matias/Escritorio/ML/test.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 17 columns):
id                         14166 non-null int64
created_on                 14166 non-null object
property_type              14166 non-null object
operation                  14166 non-null object
place_name                 14166 non-null object
place_with_parent_names    14166 non-null object
country_name               14166 non-null object
state_name                 14166 non-null object
lat-lon                    10487 non-null object
lat                        10487 non-null float64
lon                        10487 non-null float64
surface_total_in_m2        11853 non-null float64
surface_covered_in_m2      13005 non-null float64
floor                      1368 non-null float64
rooms                      7500 non-null float64
expenses                   2543 non-null object
description                14166 non-null object
dtypes: float64(6), int64(1), object(10)
memory usage: 1

In [32]:
# completamos la surface total in m2 asi ya queda completo
test['surface_total_in_m2'] = test.groupby('place_name')['surface_total_in_m2'].transform(lambda x: x.fillna(x.median()))
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 17 columns):
id                         14166 non-null int64
created_on                 14166 non-null object
property_type              14166 non-null object
operation                  14166 non-null object
place_name                 14166 non-null object
place_with_parent_names    14166 non-null object
country_name               14166 non-null object
state_name                 14166 non-null object
lat-lon                    10487 non-null object
lat                        10487 non-null float64
lon                        10487 non-null float64
surface_total_in_m2        14154 non-null float64
surface_covered_in_m2      13005 non-null float64
floor                      1368 non-null float64
rooms                      7500 non-null float64
expenses                   2543 non-null object
description                14166 non-null object
dtypes: float64(6), int64(1), object(10)
memory usage: 1

In [ ]:
# Convertimos los datos en valores numericos

In [33]:
lb_make = LabelEncoder()
train['property_type_encoded'] = lb_make.fit_transform(train['property_type'])
train['place_name_encoded'] = lb_make.fit_transform(train['place_name'])
test['property_type_encoded'] = lb_make.fit_transform(test['property_type'])
test['place_name_encoded'] = lb_make.fit_transform(test['place_name'])



In [38]:
train = train.loc[:, ['property_type_encoded','place_name_encoded','surface_total_in_m2','rooms','price']]
train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 108657 entries, 30 to 196718
Data columns (total 5 columns):
property_type_encoded    108657 non-null int64
place_name_encoded       108657 non-null int64
surface_total_in_m2      108657 non-null float64
rooms                    108657 non-null float64
price                    108657 non-null float64
dtypes: float64(3), int64(2)
memory usage: 10.0 MB


In [57]:
testData = train[100000:]
results = train[100000:]
testData = testData.loc[:, ['property_type_encoded','place_name_encoded','surface_total_in_m2','rooms']]
testData.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 8657 entries, 181158 to 196718
Data columns (total 4 columns):
property_type_encoded    8657 non-null int64
place_name_encoded       8657 non-null int64
surface_total_in_m2      8657 non-null float64
rooms                    8657 non-null float64
dtypes: float64(2), int64(2)
memory usage: 338.2 KB


In [40]:
trainData = train[0:100000]
trainData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 30 to 181157
Data columns (total 5 columns):
property_type_encoded    100000 non-null int64
place_name_encoded       100000 non-null int64
surface_total_in_m2      100000 non-null float64
rooms                    100000 non-null float64
price                    100000 non-null float64
dtypes: float64(3), int64(2)
memory usage: 4.6 MB


In [88]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(trainData.values)
    rmse= np.sqrt(-cross_val_score(model, train_no_prices.values,train_prices.values, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)




In [89]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [90]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))



Lasso score: 820957.9396 (50806.2808)



In [58]:
knn = neighbors.KNeighborsRegressor(7, weights="distance", algorithm="kd_tree")

In [59]:
train_no_prices = trainData.loc[:, ['property_type_encoded', 'place_name_encoded','surface_total_in_m2','rooms']]
train_prices = trainData.loc[:, 'price']

In [86]:
score = rmsle_cv(knn)
print("\nKNN score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))



KNN score: 602.9705 (209.7774)



In [87]:
predictions = knn.fit(train_no_prices, train_prices).predict(testData)
predictions = predictions.flatten()

In [61]:
comparacion = pd.DataFrame()
comparacion['reales'] = results['price']
comparacion['predichos'] = predictions
comparacion

,reales,predichos
181158,1457000.0,1.152748e+06
181159,1450000.0,1.152748e+06
181160,1508000.0,1.152748e+06
181161,210000.0,2.565000e+05
181162,1508000.0,1.152748e+06
181163,1508000.0,1.152748e+06
181164,1453000.0,1.152748e+06
181165,1511000.0,1.152748e+06
181166,1511000.0,1.152748e+06
181167,75000.0,4.892857e+05
